In [38]:
import pandas as pd  # 导入pandas工具包
import numpy as np  # 导入numpy工具包
data = pd.read_excel("./result/tot_last_protect.xlsx")
n = list(data.columns)
# 这里以最优值为参考数列
n.remove("Unnamed: 0")
n.remove("对人类影响")
# n.remove("对人类正收益")
data = data[n]
data
# # # 转换形式
for i in n:
    data[i] = data[i]/(np.max(data[i])-np.min(data[i]))
    
A1 = []
# 获取最优列值
for i in n:
    Max = np.max(data[i])
    A1.append(Max)
   
# 转换形式    
A1 = np.array(A1)
m = len(data)
for i in range(m):
    data[i:i+1] = abs(data[i:i+1] - A1)
    
data
    
# 最大值
MAX = []
# 每个指标的最大值
for i in n:
    l = max(data[i])
    MAX.append(l)

MAX = max(MAX)

# 最小值
MIN = []
# 每个指标的最小值
for i in n:
    l = min(data[i])
    MIN.append(l)

MIN = min(MIN)

for i in n:
    data[i] = (MIN + 0.5*MAX)/(data[i] + 0.5*MAX)
    
    
score = []
for i in range(m):
    s = sum(data[i:i+1].values[0])/len(n)
    score.append(s)

data1 = pd.DataFrame(score, index = data.index, columns = ['综合得分'])
data1.index.name = '城市'
# 排序
data1['排序'] = data1.rank(ascending = False)

# 保存为excel
data1.to_excel("./综合得分.xlsx")
data1

,综合得分,排序
城市,,
0,0.619048,3.0
1,0.676910,1.0
2,0.527092,6.0
3,0.447472,8.0
4,0.442633,9.0
5,0.625492,2.0
6,0.563271,5.0
7,0.497565,7.0
8,0.567856,4.0


In [28]:
import pandas as pd
import numpy as np
import math
from numpy import array
import copy

# 熵值法函数 求指标权重
def cal_entropy_weight(x, label = dict()):
    '''熵值法计算变量的权重'''
    # print('原始数据：'+ str(x))
    # 获得表的列头
    x_header = x.keys()
    # 初始化默认的label：1为正向指标 2 为负向指标
    if len(label) == 0:
        for key in x_header:
            label[key] = 1
    # print('label:'+str(label))
    # 标准化
    for key in label:
        # 得到列最大和最小
        maxium = np.max(x[key], axis=0)
        minium = np.min(x[key], axis=0)
        # print('maxium:' + str(maxium))
        # print('minium:' + str(minium))
        if label[key] == 1:# 正向指标
            x[key] = x.loc[:, key].apply(lambda x: ((x - minium) / (maxium - minium)))
            # x[key].apply(lambda x: ((x - maxium) / (np.max(x) - np.min(x))))
        elif label[key] == 2: # 负向指标
            x[key] = x.loc[:, key].apply(lambda x: ((maxium - x) / (maxium - minium)))
        else:
            print('输入的数据标签有误！')
            return
        # x_standardized.append(pd.DataFrame({key:temp_x}), ignore_index=True)

    # print('x_standardized:' + str(x))
    # 真向标准化
    # x = x.apply(lambda x: ((x - np.min(x)) / (np.max(x) - np.min(x))))
    # 负向标准化
    # x = x.apply(lambda x: ((np.max(x) - x) / (np.max(x) - np.min(x))))
    # 获得标准化的表中的值
    x_standardized_data = list(x.to_dict().values())
    # print('标准化：' + str(x))

    # 求k
    rows = x.index.size  # 行
    cols = x.columns.size  # 列
    k = 1.0 / math.log(rows)
    lnf = [[None] * cols for i in range(rows)]
    # 矩阵计算--
    # 信息熵
    # p=array(p)
    x = array(x)
    lnf = [[None] * cols for i in range(rows)]
    lnf = array(lnf)
    # p_arry = array()
    temp_p_list = list()
    p_row_list = list()
    for i in range(0, rows):
        for j in range(0, cols):
            if x[i][j] == 0:
                lnfij = 0.0
                p_row_list.append(0.0000)
            else:
                p = x[i][j] / x.sum(axis=0)[j]
                # print(type(p))
                p_row_list.append(p)
                lnfij = math.log(p) * p * (-k)
            lnf[i][j] = lnfij
        temp_p_list.append(copy.deepcopy(p_row_list))
        p_row_list.clear()
    # 行列转置
    p_list = [[r[col] for r in temp_p_list] for col in range(len(temp_p_list[0]))]
    p_dict = dict()
    p_num = 0
    for k in x_header:
        p_dict[k] = p_list[p_num]
        p_num += 1
    p_df = pd.DataFrame(p_dict)
    # print(p_list)
    # print('参数的熵' + str(p_df))
    lnf = pd.DataFrame(lnf)
    E = lnf
    # 计算冗余度
    d = 1 - E.sum(axis=0)
    # 计算各指标的权重
    w = [[None] * 1 for i in range(cols)]
    for j in range(0, cols):
        wj = d[j] / sum(d)
        w[j] = wj
     # 计算各样本的综合得分,用标准化后的数据   
    record_row = 0.0
    record = list()
    record_sum = 0.00
    x_standardized_data =  [[r[col] for r in x_standardized_data] for col in range(len(x_standardized_data[0]))]
    # print('转置后的标准化:'+str(x_original_data))
    for i in range(len(x_standardized_data)):
        for j in range(len(x_standardized_data[i])):
            # print('x_original_data[i][j]' + str(x_original_data[i][j]))
            # print('w:' + str(w[i]))
            record_sum += x_standardized_data[i][j]*w[j]
            record_row += x_standardized_data[i][j]*w[j]
        # print('record_row:' + str(round(record_row, 2)))
        record.append(round(record_row * 100, 2))
        record_row = 0.0
    print('record:' + str(record))
    record_average = round(record_sum/len(record)*100, 2)
    print('record_average:' + str(record_average))
    w = pd.DataFrame(w)
    return w

import pandas as pd  # 导入pandas工具包
import numpy as np  # 导入numpy工具包
data = pd.read_excel("./result/tot_last.xlsx")
data.pop("Unnamed: 0")
data.pop("对人类影响")
# data.pop("Unnamed: 9")
data = pd.DataFrame(data).to_dict(orient='list')
data


# a = {'1': 0.5537, '2': 0.5221, '3': 0.5133, '4': 0.5688, '5': 0.5498, '6': 0.5104}
# b = {'1': 0.3047, '2': 0.4371, '3': 0.3436, '4': 0.5225, '5': 1.0353, '6': 0.9051}
# c = {'1': 0.0000, '2': 0.1851, '3': -0.0288, '4': 0.3011, '5': -0.1522, '6': 0.1549}
# d = {'1': 0.3207, '2': 0.3282, '3': 0.3321, '4': 0.3365, '5': 0.3302, '6': 0.3424}
# deal_b= dict()
# for key, value in b.items():
#     deal_b[key] = abs(b[key] - 1)
# # data = {'a':list(a.values()), 'b':list(b.values()), 'c':list(c.values()), 'd':list(d.values())}
label = {'草地覆盖':1, '草地离散度':1, '食草动物数目':1, '食草动物离散度':1, '食肉动物':1, '食肉动物离散度':1, '对人类正收益':1}

# # # 求熵权法
df = pd.DataFrame(data)
df.dropna()
w = cal_entropy_weight(df, label)  # 调用cal_weight
w.index = df.columns
w.columns = ['weight']
w.to_excel("./评价.xlsx")




record:[13.5, 95.45, 35.51, 32.46, 29.93, 19.38, 22.81, 29.59, 16.46, 29.55, 36.42, 13.8, 27.7, 23.58, 83.39]
record_average:33.97
